# Получение данных по фьючерсам через Alor OpenAPI. Проверка на коинтеграцию.

In [1]:
import os
import json
from datetime import datetime
from io import StringIO

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns

## Скачивание данных через API Алор

In [2]:
def fetch_fut_data(fut_code: str) -> pd.DataFrame:
    """Функция выкачивает данные по shortcode фьючерса.
    
    Длительность таймфрейма 'tf'. В качестве значения можно указать точное количество секунд или код таймфрейма:
    15 — 15 секунд
    60 — 60 секунд или 1 минута
    3600 — 3600 секунд или 1 час
    D — сутки (соответствует значению 86400)
    W — неделя (соответствует значению 604800)
    M — месяц (соответствует значению 2592000)
    Y — год (соответствует значению 31536000)
    
    """
    
    url = "https://api.alor.ru/md/v2/history"
    
    payload = {}
    headers = {
      'Accept': 'application/json',
      'Authorization': 'Bearer <token>'
    }
    
    from_time = int(datetime(2025, 1, 1, 0, 0).timestamp())
    to_time = int(datetime(2025, 12, 31, 0, 0).timestamp())
    
    get_params = {
        'symbol': fut_code,
        'exchange': 'MOEX',
        'instrumentGroup': 'RFUD',
        'tf': 3600,
        'from': from_time,  # 
        'to': to_time,      # 1740751200,
        'splitAdjust': 'true',
        'format': 'Heavy',
        'jsonResponse': 'true',
    }
    
    response = requests.request("GET", url, headers=headers, data=payload, params=get_params)
    # data = json.loads(response.text)
    data = StringIO(response.text)
    
    return pd.json_normalize(pd.read_json(data)['history'])

In [3]:
def fetch_all_current_futures_data() -> pd.DataFrame:
    url = "https://api.alor.ru/md/v2/Securities?sector=FORTS&exchange=MOEX&instrumentGroup=RFUD&limit=100"
    
    payload = {}
    headers = {
      'Accept': 'application/json',
      # 'Authorization': 'Bearer <token>'
    }
    
    response = requests.request("GET", url, headers=headers, data=payload)
    return pd.json_normalize(json.loads(response.text))
    

In [4]:
def save_fut_data(short_code: str, fut_data: pd.DataFrame):
    
    dir_path = f"./data/{short_code}/"
    filename = short_code + ".csv"
    
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    fut_data.to_csv(dir_path + filename)

In [5]:
def save_futures_data(futures_codes: pd.Series):
    
    for code in futures_codes:
        print(f"Запрос {code}")
        _df = fetch_fut_data(code)
        print(_df.head(1))
        save_fut_data(code, _df)

    print("DONE.")
    

In [6]:
futures = fetch_all_current_futures_data()

In [7]:
futures.head()

,symbol,shortname,description,exchange,market,type,lotsize,facevalue,cfiCode,cancellation,...,currency,ISIN,yield,board,primary_board,tradingStatus,tradingStatusInfo,complexProductCategory,priceMultiplier,priceShownUnits
0,CNY-6.25,CRM5,CRM5,MOEX,FORTS,Фьючерсный контракт CNY-6.25,1,1000,FFXCSX,2025-06-19T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
1,CNYRUBF,CNYRUBF,CNYRUBF,MOEX,FORTS,CNYRUBF,1,1000,FFCCSX,2100-01-01T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
2,NG-5.25,NGK5,NGK5,MOEX,FORTS,Фьючерсный контракт NG-5.25,1,100,FCXCSX,2025-05-28T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
3,NGM-5.25,NRK5,NRK5,MOEX,FORTS,Фьючерсный контракт NGM-5.25,1,1,FCXCSX,2025-05-28T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
4,NG-6.24,NGM4,NGM4,MOEX,FORTS,Фьючерсный контракт NG-6.24,1,100,FCXCSX,2024-06-26T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,18,нет торгов или торги закрыты,2,1,1


In [8]:
futures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   symbol                  50 non-null     object 
 1   shortname               50 non-null     object 
 2   description             50 non-null     object 
 3   exchange                50 non-null     object 
 4   market                  50 non-null     object 
 5   type                    50 non-null     object 
 6   lotsize                 50 non-null     int64  
 7   facevalue               50 non-null     int64  
 8   cfiCode                 50 non-null     object 
 9   cancellation            50 non-null     object 
 10  minstep                 50 non-null     float64
 11  rating                  50 non-null     int64  
 12  marginbuy               50 non-null     float64
 13  marginsell              50 non-null     float64
 14  marginrate              50 non-null     floa

In [9]:
futures.to_csv('futures50.csv')

In [10]:
for idx, code in enumerate(futures['shortname']):
    print(idx, f". {code}...", end='')
    _df = fetch_fut_data(code)
    # print(_df.head(1))
    save_fut_data(code, _df)
    print("ok")

0 . CRM5...ok
1 . CNYRUBF...ok
2 . NGK5...ok
3 . NRK5...ok
4 . NGM4...ok
5 . SiM5...ok
6 . GLDRUBF...ok
7 . IMOEXF...ok
8 . GZM5...ok
9 . BRM5...ok
10 . MMM5...ok
11 . GLM5...ok
12 . CRU5...ok
13 . NAM5...ok
14 . SVM5...ok
15 . NGM5...ok
16 . GDM5...ok
17 . GKM5...ok
18 . CRM5CRU5...ok
19 . MXM5...ok
20 . USDRUBF...ok
21 . BRN4...ok
22 . NRM5...ok
23 . MNM5...ok
24 . CCN5...ok
25 . SRM5...ok
26 . SiM5SiU5...ok
27 . RIM5...ok
28 . SSM5...ok
29 . UCM5...ok
30 . NGK5NGM5...ok
31 . EuM5...ok
32 . EDM5...ok
33 . SiU5...ok
34 . SFM5...ok
35 . NGM4NGN4...ok
36 . TBM5...ok
37 . CMM5...ok
38 . VBM5...ok
39 . YDM5...ok
40 . SEM5...ok
41 . GAZPF...ok
42 . TNM5...ok
43 . PIM5...ok
44 . GZU5...ok
45 . BRN5...ok
46 . EuU5...ok
47 . BMM5...ok
48 . BRM5BRN5...ok
49 . BRN4BRQ4...ok


## Препроцессинг / склейка данных

**Обозначение месяца/квартала:**
- F — январь (January)
- G — февраль (February)
- **H — март (March)**
- J — апрель (April)
- K — май (May)
- **M — июнь (June)**
- N — июль (July)
- Q — август (August)
- **U — сентябрь (September)**
- V — октябрь (October)
- X — ноябрь (November)
- **Z — декабрь (December)**

Фьючерсы CHMF-3.25 на акции Северсталь. 
("CHM5", "RFUD", "CHMF-6.25")

GMKN-3.25 - Нор. никель. GKH5. GK

Фьючерсы мосбиржи:
- https://www.moex.com/ru/derivatives/equity/stocks/

Флор API
- https://alor.dev/docs/api/http/md-v-2-history-get

In [11]:
from pathlib import Path

def find_fut_csv(quart_code: str = ''):
    base_dir = Path("data")
    pattern = "*.csv" if not quart_code else f"*{quart_code}.csv"
    
    # Рекурсивный поиск всех файлов, соответствующих шаблону
    csv_paths = base_dir.rglob(pattern)
    csv_paths = list(map(str, filter(lambda path: ".ipynb" not in str(path), csv_paths)))
    
    return csv_paths

In [12]:
def merge_fut_data(fut_files: list = []):

    if not fut_files:
        fut_files = find_fut_csv()

    merged_df = None
    
    for idx, file in enumerate(fut_files):
        if idx == 0:
            merged_df = pd.read_csv(file, index_col=0).loc[:, ['time', 'close']]
            merged_df.rename(columns={"close": file.split('/')[1].lower()}, inplace=True)
            continue
            
        _df = pd.read_csv(file, index_col=0).loc[:, ['time', 'close']]
        _df.rename(columns={"close": file.split('/')[1].lower()}, inplace=True)
        
        merged_df = pd.merge(merged_df, _df, on='time', how='inner').sort_values(by='time').set_index('time')
        # print()
        # print(idx, file)
        # print(merged_df.head(5))
        
    return merged_df

In [13]:
df = merge_fut_data(find_fut_csv('M5'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 726 entries, 1739383200 to 1748289600
Data columns (total 30 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gdm5      726 non-null    float64
 1   nrm5      726 non-null    float64
 2   sem5      726 non-null    float64
 3   ssm5      726 non-null    float64
 4   mxm5      726 non-null    float64
 5   sim5      726 non-null    float64
 6   eum5      726 non-null    float64
 7   svm5      726 non-null    float64
 8   rim5      726 non-null    float64
 9   ngk5ngm5  726 non-null    float64
 10  glm5      726 non-null    float64
 11  ucm5      726 non-null    float64
 12  ydm5      726 non-null    float64
 13  sfm5      726 non-null    float64
 14  pim5      726 non-null    float64
 15  gkm5      726 non-null    float64
 16  crm5      726 non-null    float64
 17  mmm5      726 non-null    float64
 18  gzm5      726 non-null    float64
 19  brm5      726 non-null    float64
 20  edm5      726 non-nul

In [14]:
df.describe()

,gdm5,nrm5,sem5,ssm5,mxm5,sim5,eum5,svm5,rim5,ngk5ngm5,...,edm5,bmm5,srm5,vbm5,ngm5,tnm5,tbm5,nam5,mnm5,cmm5
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,...,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,3180.340083,3.949069,2699.325069,1267.873278,302099.242424,86641.078512,95687.260331,33.268388,109773.691460,0.237219,...,1.105868,66.844628,31772.045455,9195.483471,3.962344,1285.316804,3344.870523,19742.747934,4675.330579,8654.915978
std,135.629437,0.356052,335.690525,161.589332,21916.176661,3495.131504,2459.621659,1.090583,6229.220564,0.081810,...,0.026364,3.929093,1275.119133,887.100538,0.372062,38.216717,236.319059,1107.114480,354.091242,1262.846437
min,2927.500000,3.370000,1796.000000,1000.000000,268300.000000,80408.000000,91109.000000,30.130000,92140.000000,0.025000,...,1.025100,58.680000,28595.000000,6918.000000,3.336000,1205.000000,2985.000000,16763.000000,3791.000000,6075.000000
25%,3046.425000,3.661500,2465.250000,1148.000000,285856.250000,83864.250000,94163.000000,32.730000,106387.500000,0.173250,...,1.079100,64.112500,30867.250000,8612.000000,3.664250,1255.000000,3148.000000,18939.250000,4442.750000,7785.000000
50%,3208.650000,3.889500,2645.500000,1204.500000,295350.000000,86688.000000,95943.500000,33.340000,109585.000000,0.236000,...,1.115850,65.465000,31608.500000,9575.500000,3.887500,1276.000000,3270.000000,19873.500000,4635.000000,8269.000000
75%,3302.650000,4.294250,2870.750000,1361.750000,317206.250000,89211.750000,97448.000000,33.947500,113447.500000,0.314000,...,1.129000,70.915000,32652.000000,9907.750000,4.326750,1316.750000,3536.750000,20663.000000,4834.000000,9879.250000
max,3455.100000,4.782000,3497.000000,1623.000000,356600.000000,97342.000000,101111.000000,35.540000,125200.000000,0.406000,...,1.147500,76.920000,34695.000000,10962.000000,4.851000,1374.000000,3898.000000,22555.000000,5718.000000,11222.000000


In [16]:
from statsmodels.tsa.stattools import coint

# Функция для проверки коинтеграции между двумя рядами
def check_cointegration(series1, series2):
    # Выполняем тест Engle-Granger
    score, p_value, _ = coint(series1, series2)
    return p_value  # Возвращаем p-value

In [17]:
import itertools

# Список колонок
columns = df.columns

# Словарь для хранения результатов
coint_results = {}

# Перебор всех возможных пар
for col1, col2 in itertools.combinations(columns, 2):
    p_value = check_cointegration(df[col1], df[col2])
    coint_results[(col1, col2)] = p_value

# Вывод результатов
for pair, p_value in coint_results.items():
    if p_value < 0.05:
        print(f"Пара {pair[0]} / {pair[1]}:\tp-value = {round(p_value, 4)}")

Пара gdm5 / nrm5:	p-value = 0.0319
Пара gdm5 / edm5:	p-value = 0.0231
Пара gdm5 / ngm5:	p-value = 0.024
Пара nrm5 / pim5:	p-value = 0.0002
Пара nrm5 / edm5:	p-value = 0.0
Пара nrm5 / ngm5:	p-value = 0.0008
Пара sem5 / rim5:	p-value = 0.0338
Пара sem5 / pim5:	p-value = 0.0177
Пара sem5 / mmm5:	p-value = 0.006
Пара sem5 / gzm5:	p-value = 0.001
Пара sem5 / srm5:	p-value = 0.0008
Пара ssm5 / mxm5:	p-value = 0.0012
Пара ssm5 / ydm5:	p-value = 0.0327
Пара ssm5 / mmm5:	p-value = 0.0018
Пара ssm5 / gzm5:	p-value = 0.0257
Пара ssm5 / srm5:	p-value = 0.0085
Пара mxm5 / mmm5:	p-value = 0.0004
Пара mxm5 / cmm5:	p-value = 0.0287
Пара sim5 / ngk5ngm5:	p-value = 0.0042
Пара sim5 / sfm5:	p-value = 0.0278
Пара sim5 / gkm5:	p-value = 0.0148
Пара sim5 / ngm5:	p-value = 0.0066
Пара sim5 / nam5:	p-value = 0.0021
Пара sim5 / mnm5:	p-value = 0.0377
Пара sim5 / cmm5:	p-value = 0.0155
Пара eum5 / ngk5ngm5:	p-value = 0.0097
Пара eum5 / sfm5:	p-value = 0.0038
Пара eum5 / nam5:	p-value = 0.0011
Пара rim5 / pim5:	

In [18]:
futures.loc[(futures.shortname == 'GKM5') | (futures.shortname == 'GZM5') , ['symbol', 'type'] ]

,symbol,type
8,GAZR-6.25,Фьючерсный контракт GAZR-6.25
17,GMKN-6.25,Фьючерсный контракт GMKN-6.25


In [19]:
futures.head()

,symbol,shortname,description,exchange,market,type,lotsize,facevalue,cfiCode,cancellation,...,currency,ISIN,yield,board,primary_board,tradingStatus,tradingStatusInfo,complexProductCategory,priceMultiplier,priceShownUnits
0,CNY-6.25,CRM5,CRM5,MOEX,FORTS,Фьючерсный контракт CNY-6.25,1,1000,FFXCSX,2025-06-19T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
1,CNYRUBF,CNYRUBF,CNYRUBF,MOEX,FORTS,CNYRUBF,1,1000,FFCCSX,2100-01-01T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
2,NG-5.25,NGK5,NGK5,MOEX,FORTS,Фьючерсный контракт NG-5.25,1,100,FCXCSX,2025-05-28T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
3,NGM-5.25,NRK5,NRK5,MOEX,FORTS,Фьючерсный контракт NGM-5.25,1,1,FCXCSX,2025-05-28T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
4,NG-6.24,NGM4,NGM4,MOEX,FORTS,Фьючерсный контракт NG-6.24,1,100,FCXCSX,2024-06-26T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,18,нет торгов или торги закрыты,2,1,1
